In [1]:
import pandas as  pd 
import numpy as np 
import requests, zipfile

In [2]:
def download(file):
  #the url and auth_value from the website 
  url = f"https://api.zindi.africa/v1/competitions/next-gen-wifi-throughput-prediction-challenge/files/{file}"
  myobj = {'auth_token': "x7nYdKyCrmLZfSkD3GJBxNkQ"} 

  x = requests.post(url, data = myobj,stream=True)
  target_path = file

  handle = open(target_path, "wb")
  for chunk in x.iter_content(chunk_size=512):
      if chunk:  # filter out keep-alive new chunks
          handle.write(chunk)
  handle.close()

In [3]:
download('Test.csv')
download('Train.csv')
download('TrainTargets.csv')
download('SampleSubmission.csv')

In [4]:
def preprocess(data,is_train=True):

  dep=data['Deployment'].unique()
  rows=[]
  for dd in dep:
    df=data[data.Deployment==dd]
    
    toxp_map=dict()
    rssi_map=dict()
    flag=0
    for i, j in df.iterrows():
        if('STA' in j[0]):
          rssi_map[j[0]]={f'AP{i}':j[i] for i in range(1,10)}
        else:
          toxp_map[j[0]]={(j.index[i],'STA'+str(int(j[i]))) for i in range(1,10)}

    
    for exp,elems in toxp_map.items():
      rows1=[]
      for e in elems:
        row=[dd,exp]
        if(e[-1]!='STA0'):
          row.extend([e[0],e[1],rssi_map[e[-1]][e[0]]])
          rows.append(row)
  

      #rows.extend(rows1)
    
  data=pd.DataFrame(rows)
  data.columns = ['Deployment','TXOP','mainAP','mainSTA','mainRSSI']
  data['ID']='Deployment'+data['Deployment'].astype(str)+'_'+data['TXOP']+'_'+data['mainAP']+'_'+data['mainSTA']
  if(is_train):
    tt=pd.read_csv('TrainTargets.csv')
    data=pd.merge(data,tt,on='ID')
  
  
  return data
  






In [5]:
train=pd.read_csv('Train.csv')
train=preprocess(train)

In [6]:
train

,Deployment,TXOP,mainAP,mainSTA,mainRSSI,ID,Target
0,9391,TXOP1,AP2,STA2,-67.19,Deployment9391_TXOP1_AP2_STA2,0.0
1,9391,TXOP1,AP7,STA7,-63.71,Deployment9391_TXOP1_AP7_STA7,108.0
2,9391,TXOP1,AP1,STA1,-64.87,Deployment9391_TXOP1_AP1_STA1,30.0
3,9391,TXOP1,AP5,STA5,-66.72,Deployment9391_TXOP1_AP5_STA5,54.0
4,9391,TXOP2,AP6,STA6,-65.12,Deployment9391_TXOP2_AP6_STA6,156.0
...,...,...,...,...,...,...,...
1749591,3934,TXOP99,AP1,STA2,-65.12,Deployment3934_TXOP99_AP1_STA2,312.0
1749592,3934,TXOP100,AP8,STA15,-67.62,Deployment3934_TXOP100_AP8_STA15,0.0
1749593,3934,TXOP100,AP2,STA3,-52.57,Deployment3934_TXOP100_AP2_STA3,156.0
1749594,3934,TXOP100,AP5,STA9,-64.87,Deployment3934_TXOP100_AP5_STA9,0.0


In [7]:
test=pd.read_csv('Test.csv')
test=preprocess(test,False)

In [8]:
test

,Deployment,TXOP,mainAP,mainSTA,mainRSSI,ID
0,9373,TXOP1,AP9,STA35,-69.12,Deployment9373_TXOP1_AP9_STA35
1,9373,TXOP2,AP1,STA3,-67.19,Deployment9373_TXOP2_AP1_STA3
2,9373,TXOP2,AP3,STA9,-63.71,Deployment9373_TXOP2_AP3_STA9
3,9373,TXOP2,AP4,STA13,-64.87,Deployment9373_TXOP2_AP4_STA13
4,9373,TXOP3,AP4,STA14,-65.58,Deployment9373_TXOP3_AP4_STA14
...,...,...,...,...,...,...
749919,5144,TXOP98,AP5,STA10,-67.62,Deployment5144_TXOP98_AP5_STA10
749920,5144,TXOP99,AP3,STA5,-63.71,Deployment5144_TXOP99_AP3_STA5
749921,5144,TXOP99,AP9,STA17,-65.58,Deployment5144_TXOP99_AP9_STA17
749922,5144,TXOP100,AP8,STA16,-64.61,Deployment5144_TXOP100_AP8_STA16


In [10]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse


folds = KFold(n_splits = 5)

predictions = []
losses = []
X=train[['mainRSSI']]
y=train['Target']
X_ts=test[['mainRSSI']]

for i,( train_index, test_index) in enumerate(folds.split(X, y)):
  print(50*'--')
  print(f'Fold {i+1}/5')
  print(50*'--')
  X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

  # Train model
  model =LGBMRegressor(n_estimators=1000)
  model.fit(X_train,y_train)
  # Make predictions
  preds = model.predict(X_ts)
  y_pred = model.predict(X_test)
  # Append predictions and losses
  predictions.append(preds)
  loss = mse(y_test, y_pred,squared=False)
  # Print loss
  print(f'{i+1}:  {loss}\n')
  losses.append(loss)
  
print(f'Mean loss  {np.mean(losses)}')

----------------------------------------------------------------------------------------------------
Fold 1/5
----------------------------------------------------------------------------------------------------
1:  86.28696925882939

----------------------------------------------------------------------------------------------------
Fold 2/5
----------------------------------------------------------------------------------------------------
2:  86.40892571217763

----------------------------------------------------------------------------------------------------
Fold 3/5
----------------------------------------------------------------------------------------------------
3:  86.44284373162247

----------------------------------------------------------------------------------------------------
Fold 4/5
----------------------------------------------------------------------------------------------------
4:  86.23226568054274

----------------------------------------------------------------

In [11]:
#Making submission
ss=test[['ID']]
ss['Target']=np.mean(predictions,0)
ss

<ipython-input-11-e0c206ac8ddd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss['Target']=np.mean(predictions,0)


,ID,Target
0,Deployment9373_TXOP1_AP9_STA35,68.565325
1,Deployment9373_TXOP2_AP1_STA3,79.698668
2,Deployment9373_TXOP2_AP3_STA9,114.407521
3,Deployment9373_TXOP2_AP4_STA13,101.279049
4,Deployment9373_TXOP3_AP4_STA14,89.033458
...,...,...
749919,Deployment5144_TXOP98_AP5_STA10,73.680251
749920,Deployment5144_TXOP99_AP3_STA5,114.407521
749921,Deployment5144_TXOP99_AP9_STA17,89.033458
749922,Deployment5144_TXOP100_AP8_STA16,102.776802


In [12]:
ss['Target']=ss['Target'].astype(np.float16)#Otherwise file will be too large

<ipython-input-12-0be4644114fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss['Target']=ss['Target'].astype(np.float16)#Otherwise file will be too large


In [13]:
ss.to_csv('ss.csv',index=False)

In [14]:
ss

,ID,Target
0,Deployment9373_TXOP1_AP9_STA35,68.5625
1,Deployment9373_TXOP2_AP1_STA3,79.6875
2,Deployment9373_TXOP2_AP3_STA9,114.4375
3,Deployment9373_TXOP2_AP4_STA13,101.2500
4,Deployment9373_TXOP3_AP4_STA14,89.0625
...,...,...
749919,Deployment5144_TXOP98_AP5_STA10,73.6875
749920,Deployment5144_TXOP99_AP3_STA5,114.4375
749921,Deployment5144_TXOP99_AP9_STA17,89.0625
749922,Deployment5144_TXOP100_AP8_STA16,102.7500
